In [ ]:
!pip install datasets pandas pyarrow


In [ ]:
import re
import os
from datasets import load_dataset
import pandas as pd


In [ ]:
from datasets import load_dataset

# Directly stream the  Gujarati split
gujarati_stream = load_dataset(
    "ai4bharat/IndicCorpV2",
    "indiccorp_v2",
    streaming=True,
    split="guj_Gujr"
)



In [ ]:
def custom_tokenizer_guj_with_matras(text):
    # Handle special items
    url_pattern = r'https?://[^\s]+|www\.[^\s]+'
    email_pattern = r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b'

    urls = re.findall(url_pattern, text)
    text = re.sub(url_pattern, '<URL>', text)

    emails = re.findall(email_pattern, text)
    text = re.sub(email_pattern, '<EMAIL>', text)

    tokens = []
    for word in text.split():
        if word == '<URL>':
            tokens.append(urls.pop(0))
        elif word == '<EMAIL>':
            tokens.append(emails.pop(0))
        else:
            # Tokenize gujarati characters while separating matras
            word_tokens = re.findall(rf'[\u0915-\u0939][\u093e-\u094c\u0902\u0903]?|\d+|[^\s\w]', word)
            tokens.extend(word_tokens)

    return tokens


In [ ]:
from itertools import islice

# Example: Get first 5 entries from the iterable
samples = list(islice(gujarati_stream, 5))  # change 5 to any number


In [ ]:
text = samples[0]['text']

In [ ]:
import re

def guj_tokenizer(text):
    url_pattern = r'https?://[^\s]+|www\.[^\s]+'
    email_pattern = r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b'

    # Save and replace URLs and emails
    urls = re.findall(url_pattern, text)
    text = re.sub(url_pattern, '<URL>', text)

    emails = re.findall(email_pattern, text)
    text = re.sub(email_pattern, '<EMAIL>', text)

    tokens = []
    for token in text.split():
        if token == '<URL>':
            tokens.append(urls.pop(0))
        elif token == '<EMAIL>':
            tokens.append(emails.pop(0))
        else:
            split_tokens = re.findall(
                r'[\u0900-\u097F]+|[a-zA-Z0-9]+|[।.,!?;:()\"\'\-]|[^\s]',
                token
            )
            tokens.extend(split_tokens)

    return tokens

def guj_sentence_tokenizer(text):
    sentence_end_pattern = r'(?<=[।!?\.])\s+'
    sentences = re.split(sentence_end_pattern, text.strip())
    return [s.strip() for s in sentences if s.strip()]

def detokenize(tokens):
    # Rebuild sentence with proper spacing logic
    sentence = ''
    for i, token in enumerate(tokens):
        if i > 0 and not re.match(r'[।.,!?;:)\]\'\"]', token):
            sentence += ' '
        sentence += token
    return sentence.strip()

def guj_corpus_statistics(text):
    sentences = guj_sentence_tokenizer(text)
    all_tokens = []
    reconstructed_sentences = []

    for sentence in sentences:
        tokens = guj_tokenizer(sentence)
        all_tokens.extend(tokens)

        # For checking reformation
        reconstructed = detokenize(tokens)
        reconstructed_sentences.append(reconstructed)

    num_tokens = len(all_tokens)
    unique_tokens = set(all_tokens)
    total_chars = sum(len(token) for token in all_tokens)

    word_tokens = [t for t in all_tokens if re.match(r'^[\u0900-\u097F\w]+$', t)]
    avg_word_length = sum(len(t) for t in word_tokens) / len(word_tokens) if word_tokens else 0
    type_token_ratio = len(unique_tokens) / num_tokens if num_tokens else 0

    return {
        'sentences': sentences,
        'tokens': all_tokens,
        'num_tokens': num_tokens,
        'total_characters': total_chars,
        'average_word_length': round(avg_word_length, 2),
        'type_token_ratio': round(type_token_ratio, 3),
        'reconstructed_sentences': reconstructed_sentences
    }


In [ ]:
text = ' '.join(sample['text'] for sample in samples)
stats = guj_corpus_statistics(text)

print("Original Sentences:")
for sentence in stats['sentences']:
    print(sentence)

print("\nReconstructed Sentences:")
for sent in stats['reconstructed_sentences']:
    print(sent)

print("\nTokens:", stats['tokens'])
print("Number of Tokens:", stats['num_tokens'])
print("Total Characters:", stats['total_characters'])
print("Average Word Length:", stats['average_word_length'])
print("Type-Token Ratio:", stats['type_token_ratio'])


Original Sentences:
આ વીડિયો જુઓ: ઊંઝા માર્કેટયાર્ડ આજથી 25 જુલાઈ સુધી બંધ  મિથેનોલ આવ્યો ક્યાંથી?
આખરે ત્રણ રાજ્યોમાં મળેલ હાર પર કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધી દ્વારા પ્રથમ પ્રતિક્રિયા આપવામાં આવી છે.
તેમણે કહ્યું કે, ત્રિપુરા, નાગાલેન્ડ અને મેઘાલયમાં લોકોના જનાદેશનો સ્વાગત કરીએ છે અને આ ક્ષેત્રના લોકોનો વિશ્વાસ ફરીથી જીતીવા માટે પ્રતિબદ્ધ છીએ.

Reconstructed Sentences:
આ વ ી ડ િ ય ો જ ુ ઓ: ઊ ં ઝ ા મ ા ર ્ ક ે ટ ય ા ર ્ ડ આ જ થ ી 25 જ ુ લ ા ઈ સ ુ ધ ી બ ં ધ મ િ થ ે ન ો લ આ વ ્ ય ો ક ્ ય ા ં થ ી?
આ ખ ર ે ત ્ ર ણ ર ા જ ્ ય ો મ ા ં મ ળ ે લ હ ા ર પ ર ક ો ં ગ ્ ર ે સ અ ધ ્ ય ક ્ ષ ર ા હ ુ લ ગ ા ં ધ ી દ ્ વ ા ર ા પ ્ ર થ મ પ ્ ર ત િ ક ્ ર િ ય ા આ પ વ ા મ ા ં આ વ ી છ ે.
ત ે મ ણ ે ક હ ્ ય ુ ં ક ે, ત ્ ર િ પ ુ ર ા, ન ા ગ ા લ ે ન ્ ડ અ ન ે મ ે ઘ ા લ ય મ ા ં લ ો ક ો ન ા જ ન ા દ ે શ ન ો સ ્ વ ા ગ ત ક ર ી એ છ ે અ ન ે આ ક ્ ષ ે ત ્ ર ન ા લ ો ક ો ન ો વ િ શ ્ વ ા સ ફ ર ી થ ી જ ી ત ી વ ા મ ા ટ ે પ ્ ર ત િ બ દ ્ ધ છ ી એ.

Tokens: ['આ', 'વ', 'ી', 'ડ', 'િ', 'ય', 'ો', 'જ', 'ુ', 'ઓ', ':', 'ઊ', 'ં', 'ઝ', 'ા',

In [ ]:
import re

def english_tokenizer(text):
    # Define patterns
    url_pattern = r'https?://[^\s]+|www\.[^\s]+'
    email_pattern = r'\b[\w\.-]+@[\w\.-]+\.\w{2,}\b'

    # Save and replace URLs/emails
    urls = re.findall(url_pattern, text)
    text = re.sub(url_pattern, '<URL>', text)

    emails = re.findall(email_pattern, text)
    text = re.sub(email_pattern, '<EMAIL>', text)

    tokens = []
    for token in text.split():
        if token == '<URL>':
            tokens.append(urls.pop(0))
        elif token == '<EMAIL>':
            tokens.append(emails.pop(0))
        else:
            # Tokenize: words, numbers, punctuation
            split_tokens = re.findall(r"[a-zA-Z0-9]+|[.,!?;:'\"()\-]|[^\s]", token)
            tokens.extend(split_tokens)

    return tokens

def english_sentence_tokenizer(text):
    # Split on sentence-ending punctuation followed by space
    sentence_end_pattern = r'(?<=[.!?])\s+'
    sentences = re.split(sentence_end_pattern, text.strip())
    return [s.strip() for s in sentences if s.strip()]

def detokenize(tokens):
    sentence = ''
    for i, token in enumerate(tokens):
        if i > 0 and not re.match(r'[.,!?;:)\]\'\"]', token):
            sentence += ' '
        sentence += token
    return sentence.strip()

def english_corpus_statistics(text):
    sentences = english_sentence_tokenizer(text)
    all_tokens = []
    reconstructed_sentences = []

    for sentence in sentences:
        tokens = english_tokenizer(sentence)
        all_tokens.extend(tokens)

        reconstructed = detokenize(tokens)
        reconstructed_sentences.append(reconstructed)

    num_tokens = len(all_tokens)
    unique_tokens = set(all_tokens)
    total_chars = sum(len(token) for token in all_tokens)

    word_tokens = [t for t in all_tokens if re.match(r'^[a-zA-Z0-9]+$', t)]
    avg_word_length = sum(len(t) for t in word_tokens) / len(word_tokens) if word_tokens else 0
    type_token_ratio = len(unique_tokens) / num_tokens if num_tokens else 0

    return {
        'sentences': sentences,
        'tokens': all_tokens,
        'num_tokens': num_tokens,
        'total_characters': total_chars,
        'average_word_length': round(avg_word_length, 2),
        'type_token_ratio': round(type_token_ratio, 3),
        'reconstructed_sentences': reconstructed_sentences
    }


In [ ]:
text = """Hello! I'm Alex. Feel free to drop a message at alex.jordan@gmail.com or visit my blog at www.alexwrites.com. Hope you find it interesting!"""

stats = english_corpus_statistics(text)

print("Original Sentences:")
for s in stats['sentences']:
    print(s)

print("\nReconstructed Sentences:")
for s in stats['reconstructed_sentences']:
    print(s)

print("\nTokens:", stats['tokens'])
print("Number of Tokens:", stats['num_tokens'])
print("Total Characters:", stats['total_characters'])
print("Average Word Length:", stats['average_word_length'])
print("Type-Token Ratio:", stats['type_token_ratio'])


Original Sentences:
Hello!
I'm Alex.
Feel free to drop a message at alex.jordan@gmail.com or visit my blog at www.alexwrites.com.
Hope you find it interesting!

Reconstructed Sentences:
Hello!
I' m Alex.
Feel free to drop a message at alex.jordan@gmail.com or visit my blog at www.alexwrites.com.
Hope you find it interesting!

Tokens: ['Hello', '!', 'I', "'", 'm', 'Alex', '.', 'Feel', 'free', 'to', 'drop', 'a', 'message', 'at', 'alex.jordan@gmail.com', 'or', 'visit', 'my', 'blog', 'at', 'www.alexwrites.com.', 'Hope', 'you', 'find', 'it', 'interesting', '!']
Number of Tokens: 27
Total Characters: 118
Average Word Length: 3.52
Type-Token Ratio: 0.926
